In [1]:
# from flair.models import TextClassifier
# from flair.data import Sentence
# classifier = TextClassifier.load('en-sentiment')
# sentence = Sentence('ache is bad!')
# classifier.predict(sentence)
# # print sentence with predicted labels
# print('Sentence above is: ', sentence.labels)

In [2]:
import pandas as pd
import numpy as np
import itertools
import codecs
import os

In [3]:
# get the current working directory
data_path = os.getcwd()+'/Datasets/'
# os.path.abspath(os.path.join(os.path.dirname( '__file__' ), '..', 'Datasets'))+'/'

# Although the dataset says csv, it is tab delimited. In addition to this, they have severe codels problems. 
# So best to parse throught codes first. 
# UnicodeDecodeError: 'utf-8' codec can't decode byte 0xfc in position 2: invalid start byte

#open for reading with "universal" type set

import codecs

doc_d_t = codecs.open(data_path+'EUADR_Corpus_IBIgroup/'+'EUADR_drug_target'+'.csv','rU','UTF-8') 
EUADR_drug_target = pd.read_csv(doc_d_t, sep='\t', na_filter = False)
EUADR_drug_target['CLASS'] = 'drug_gene'

doc_t_d = codecs.open(data_path+'EUADR_Corpus_IBIgroup/'+'EUADR_target_disease'+'.csv','rU','UTF-8',errors='ignore') 
EUADR_target_disease = pd.read_csv(doc_t_d, sep='\t', na_filter = False)
EUADR_target_disease['CLASS'] = 'gene_disease'
       
doc_d_d = codecs.open(data_path+'EUADR_Corpus_IBIgroup/'+'EUADR_drug_disease'+'.csv','rU','UTF-8')                       
EUADR_drug_disease = pd.read_csv(doc_d_d, sep='\t', na_filter = False)
EUADR_drug_disease['CLASS'] = 'drug_disease'


EUADR_temp = EUADR_drug_target.append(EUADR_target_disease).append(EUADR_drug_disease)

EUADR_temp_1 = EUADR_temp[EUADR_temp['ASSOCIATION_TYPE'] == 'PA']
EUADR_temp_2 = EUADR_temp[EUADR_temp['ASSOCIATION_TYPE'] == 'NA']
EUADR_temp = EUADR_temp_1.append(EUADR_temp_2)
EUADR_temp.rename(columns={"SENTENCE": "text", "ASSOCIATION_TYPE": "label"}, inplace=True)

EUADR_temp.reset_index(inplace=True)

In [4]:
# Get GAD dataset

doc_t_d = codecs.open(data_path+'GAD_Corpus_IBIgroup/'+'GAD_Y_N'+'.csv','rU','UTF-8',errors='ignore') 
GAD_target_disease_Y_N = pd.read_csv(doc_t_d, sep='\t', na_filter = False)
GAD_target_disease_Y_N['CLASS'] = 'gene_disease'

doc_t_d = codecs.open(data_path+'GAD_Corpus_IBIgroup/'+'GAD_F'+'.csv','rU','UTF-8',errors='ignore') 
GAD_target_disease_F = pd.read_csv(doc_t_d, sep='\t', na_filter = False)
GAD_target_disease_F['CLASS'] = 'gene_disease'

GAD_temp = GAD_target_disease_Y_N



GAD_temp['GAD_ASSOC'] = GAD_temp['GAD_ASSOC'].apply(lambda x: x.replace('Y', 'PA').replace('N', 'NA').replace('F', 'FA'))

GAD_temp.rename(columns={"GAD_ASSOC": "label", "GAD_CONCLUSION": "text"}, inplace=True)
GAD_temp.reset_index(inplace=True)

In [5]:
df = EUADR_temp[['label','text']].append(GAD_temp[['label','text']])
df.reset_index(inplace=True)

In [6]:
df.drop("index", axis=1, inplace=True)


In [7]:
data_loc = data_path+'EU-ADR+GAD/'

data = df
data['label'] = '__label__' + data['label'].astype(str)
data.iloc[0:int(len(data)*0.8)].to_csv(data_loc+'train.csv', sep='\t', index = False, header = False)
data.iloc[int(len(data)*0.8):int(len(data)*0.9)].to_csv(data_loc+'test.csv', sep='\t', index = False, header = False)
data.iloc[int(len(data)*0.9):].to_csv(data_loc+'dev.csv', sep='\t', index = False, header = False);


In [16]:
from flair.data_fetcher import NLPTaskDataFetcher
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentRNNEmbeddings
from flair.embeddings import TokenEmbeddings, PooledFlairEmbeddings, StackedEmbeddings, CharacterEmbeddings
from flair.embeddings import BertEmbeddings
from flair.embeddings import ELMoEmbeddings
from flair.models import TextClassifier
from flair.trainers import ModelTrainer
from pathlib import Path
from typing import List


elmo_embeddings_pubmed = ELMoEmbeddings('pubmed')


100%|██████████| 374434792/374434792 [00:10<00:00, 34084607.40B/s]


In [17]:
corpus = NLPTaskDataFetcher.load_classification_corpus(Path(data_loc), test_file='test.csv', dev_file='dev.csv', train_file='train.csv')


2019-09-11 13:05:43,271 Reading data from /nfs/gns/literature/Santosh_Tirunagari/GitHub/Unsupervised-Protein-Genes-Diseases-Extraction/Datasets/EU-ADR+GAD
2019-09-11 13:05:43,274 Train: /nfs/gns/literature/Santosh_Tirunagari/GitHub/Unsupervised-Protein-Genes-Diseases-Extraction/Datasets/EU-ADR+GAD/train.csv
2019-09-11 13:05:43,275 Dev: /nfs/gns/literature/Santosh_Tirunagari/GitHub/Unsupervised-Protein-Genes-Diseases-Extraction/Datasets/EU-ADR+GAD/dev.csv
2019-09-11 13:05:43,277 Test: /nfs/gns/literature/Santosh_Tirunagari/GitHub/Unsupervised-Protein-Genes-Diseases-Extraction/Datasets/EU-ADR+GAD/test.csv


/homes/stirunag/anaconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated function (or staticmethod) load_classification_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  """Entry point for launching an IPython kernel.
/homes/stirunag/anaconda3/envs/pytorch/lib/python3.7/site-packages/flair/data_fetcher.py:447: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  max_tokens_per_doc=max_tokens_per_doc,
/homes/stirunag/anaconda3/envs/pytorch/lib/python3.7/site-packages/flair/data_fetcher.py:454: DeprecationWarning: Call to deprecated function (or staticmethod) read_text_classification_file. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  max_tokens_per_doc=max_tokens_per_doc,
/homes/stirunag/anaconda3/envs/pytorch/lib/python3.7/site-packages/flair/data_fetcher.py:463: Depre

In [18]:
# 3. make a list of word embeddings
word_embeddings = [WordEmbeddings('glove'),

                   # comment in flair embeddings for state-of-the-art results
                   FlairEmbeddings('news-forward'),
                   FlairEmbeddings('news-backward'),
                   CharacterEmbeddings(), 
                   PooledFlairEmbeddings('pubmed-backward', pooling='max'),
                   PooledFlairEmbeddings('pubmed-forward', pooling='max'),
                   # BertEmbedding('bert-large-cased'),
                   elmo_embeddings_pubmed
                   ]



In [19]:
# document_embeddings = DocumentRNNEmbeddings(word_embeddings, hidden_size=512, reproject_words=True, reproject_words_dimension=256)
document_embeddings: DocumentRNNEmbeddings = DocumentRNNEmbeddings(word_embeddings,
                                                                   rnn_type = "LSTM",
                                                                     hidden_size=512,
                                                                     reproject_words=True,
                                                                     reproject_words_dimension=256,
                                                                     )

In [20]:
classifier = TextClassifier(document_embeddings, label_dictionary=corpus.make_label_dictionary(), multi_label=False)



2019-09-11 13:06:20,740 Computing label dictionary. Progress:


100%|██████████| 2688/2688 [00:00<00:00, 413068.41it/s]

2019-09-11 13:06:20,752 [b'PA', b'NA']


In [21]:
trainer = ModelTrainer(classifier, corpus)


In [22]:
# 7. start the training
# trainer.train(data_loc, max_epochs=10)

result_loc = '../flair_models/text_classification/sentiment_model'
trainer.train(result_loc,
              learning_rate=0.15,
              mini_batch_size=128,
              anneal_factor=0.5,
              patience=5,
              max_epochs=150)

2019-09-11 13:06:31,583 ----------------------------------------------------------------------------------------------------
2019-09-11 13:06:31,584 Model: "TextClassifier(
  (document_embeddings): DocumentRNNEmbeddings(
    (embeddings): StackedEmbeddings(
      (list_embedding_0): WordEmbeddings('glove')
      (list_embedding_1): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.05)
          (encoder): Embedding(300, 100)
          (rnn): LSTM(100, 2048)
          (decoder): Linear(in_features=2048, out_features=300, bias=True)
        )
      )
      (list_embedding_2): FlairEmbeddings(
        (lm): LanguageModel(
          (drop): Dropout(p=0.05)
          (encoder): Embedding(300, 100)
          (rnn): LSTM(100, 2048)
          (decoder): Linear(in_features=2048, out_features=300, bias=True)
        )
      )
      (list_embedding_3): CharacterEmbeddings(
        (char_embedding): Embedding(275, 25)
        (char_rnn): LSTM(25, 25, bidirectional=True)
 

2019-09-11 13:14:03,499 epoch 4 - iter 10/21 - loss 0.41926819 - samples/sec: 37.90
2019-09-11 13:14:14,073 epoch 4 - iter 12/21 - loss 0.39913638 - samples/sec: 31.75
2019-09-11 13:14:23,939 epoch 4 - iter 14/21 - loss 0.38741976 - samples/sec: 36.30
2019-09-11 13:14:32,651 epoch 4 - iter 16/21 - loss 0.37411225 - samples/sec: 37.26
2019-09-11 13:14:41,188 epoch 4 - iter 18/21 - loss 0.38246225 - samples/sec: 38.31
2019-09-11 13:14:52,261 epoch 4 - iter 20/21 - loss 0.39112726 - samples/sec: 28.30
2019-09-11 13:14:54,698 ----------------------------------------------------------------------------------------------------
2019-09-11 13:14:54,700 EPOCH 4 done: loss 0.3911 - lr 0.1500
2019-09-11 13:14:58,371 DEV : loss 0.2850567698478699 - score 0.9077
2019-09-11 13:14:58,470 BAD EPOCHS (no improvement): 1
2019-09-11 13:14:58,477 ----------------------------------------------------------------------------------------------------
train mode resetting embeddings
train mode resetting embeddi

2019-09-11 13:23:44,078 epoch 10 - iter 2/21 - loss 0.17251630 - samples/sec: 40.40
2019-09-11 13:23:53,161 epoch 10 - iter 4/21 - loss 0.17140375 - samples/sec: 40.52
2019-09-11 13:24:02,046 epoch 10 - iter 6/21 - loss 0.17047549 - samples/sec: 39.86
2019-09-11 13:24:12,318 epoch 10 - iter 8/21 - loss 0.16350310 - samples/sec: 37.11
2019-09-11 13:24:21,596 epoch 10 - iter 10/21 - loss 0.15786464 - samples/sec: 36.78
2019-09-11 13:24:31,240 epoch 10 - iter 12/21 - loss 0.15338021 - samples/sec: 36.71
2019-09-11 13:24:40,488 epoch 10 - iter 14/21 - loss 0.15590387 - samples/sec: 37.34
2019-09-11 13:24:49,467 epoch 10 - iter 16/21 - loss 0.15734967 - samples/sec: 37.55
2019-09-11 13:24:59,126 epoch 10 - iter 18/21 - loss 0.16273786 - samples/sec: 37.60
2019-09-11 13:25:09,676 epoch 10 - iter 20/21 - loss 0.15876813 - samples/sec: 28.56
2019-09-11 13:25:11,917 ----------------------------------------------------------------------------------------------------
2019-09-11 13:25:11,919 EPOCH

Epoch    14: reducing learning rate of group 0 to 3.7500e-02.
2019-09-11 13:33:43,103 BAD EPOCHS (no improvement): 6
2019-09-11 13:33:43,108 ----------------------------------------------------------------------------------------------------
train mode resetting embeddings
train mode resetting embeddings
2019-09-11 13:33:46,745 epoch 16 - iter 0/21 - loss 0.06376326 - samples/sec: 72.23
2019-09-11 13:33:56,065 epoch 16 - iter 2/21 - loss 0.09020484 - samples/sec: 36.60
2019-09-11 13:34:04,558 epoch 16 - iter 4/21 - loss 0.09968677 - samples/sec: 37.98
2019-09-11 13:34:13,655 epoch 16 - iter 6/21 - loss 0.10068461 - samples/sec: 37.99
2019-09-11 13:34:23,071 epoch 16 - iter 8/21 - loss 0.10439900 - samples/sec: 35.16
2019-09-11 13:34:33,020 epoch 16 - iter 10/21 - loss 0.10720378 - samples/sec: 38.65
2019-09-11 13:34:43,119 epoch 16 - iter 12/21 - loss 0.11049270 - samples/sec: 39.62
2019-09-11 13:34:52,327 epoch 16 - iter 14/21 - loss 0.12003084 - samples/sec: 37.43
2019-09-11 13:35:02

2019-09-11 13:44:00,219 epoch 21 - iter 18/21 - loss 0.11027539 - samples/sec: 39.09
2019-09-11 13:44:11,474 epoch 21 - iter 20/21 - loss 0.11131524 - samples/sec: 28.76
2019-09-11 13:44:13,601 ----------------------------------------------------------------------------------------------------
2019-09-11 13:44:13,603 EPOCH 21 done: loss 0.1113 - lr 0.0375
2019-09-11 13:44:17,219 DEV : loss 0.3175256550312042 - score 0.9077
Epoch    20: reducing learning rate of group 0 to 1.8750e-02.
2019-09-11 13:44:17,318 BAD EPOCHS (no improvement): 6
2019-09-11 13:44:17,337 ----------------------------------------------------------------------------------------------------
train mode resetting embeddings
train mode resetting embeddings
2019-09-11 13:44:21,267 epoch 22 - iter 0/21 - loss 0.08292674 - samples/sec: 66.51
2019-09-11 13:44:31,034 epoch 22 - iter 2/21 - loss 0.09416270 - samples/sec: 37.87
2019-09-11 13:44:40,396 epoch 22 - iter 4/21 - loss 0.09734023 - samples/sec: 38.85
2019-09-11 13:4

...

2019-09-11 14:31:31,292 epoch 50 - iter 2/21 - loss 0.11205126 - samples/sec: 37.63
2019-09-11 14:31:40,627 epoch 50 - iter 4/21 - loss 0.09781258 - samples/sec: 35.81
2019-09-11 14:31:48,884 epoch 50 - iter 6/21 - loss 0.09486714 - samples/sec: 40.31
2019-09-11 14:31:57,851 epoch 50 - iter 8/21 - loss 0.09658214 - samples/sec: 37.68
2019-09-11 14:32:07,493 epoch 50 - iter 10/21 - loss 0.10341543 - samples/sec: 39.35
2019-09-11 14:32:16,626 epoch 50 - iter 12/21 - loss 0.09324966 - samples/sec: 39.55
2019-09-11 14:32:25,572 epoch 50 - iter 14/21 - loss 0.09237952 - samples/sec: 38.74
2019-09-11 14:32:33,844 epoch 50 - iter 16/21 - loss 0.08925688 - samples/sec: 38.19
2019-09-11 14:32:43,009 epoch 50 - iter 18/21 - loss 0.08920468 - samples/sec: 40.44
2019-09-11 14:32:53,381 epoch 50 - iter 20/21 - loss 0.08776829 - samples/sec: 30.11
2019-09-11 14:32:55,737 ----------------------------------------------------------------------------------------------------
2019-09-11 14:32:55,739 EPOCH

2019-09-11 14:41:33,842 DEV : loss 0.34076353907585144 - score 0.9048
2019-09-11 14:41:33,938 BAD EPOCHS (no improvement): 4
2019-09-11 14:41:33,942 ----------------------------------------------------------------------------------------------------
train mode resetting embeddings
train mode resetting embeddings
2019-09-11 14:41:37,571 epoch 56 - iter 0/21 - loss 0.12394780 - samples/sec: 72.30
2019-09-11 14:41:46,673 epoch 56 - iter 2/21 - loss 0.07875582 - samples/sec: 40.24
2019-09-11 14:41:55,911 epoch 56 - iter 4/21 - loss 0.08305984 - samples/sec: 38.58
2019-09-11 14:42:04,132 epoch 56 - iter 6/21 - loss 0.08882976 - samples/sec: 40.62
2019-09-11 14:42:13,217 epoch 56 - iter 8/21 - loss 0.08025794 - samples/sec: 40.61
2019-09-11 14:42:22,014 epoch 56 - iter 10/21 - loss 0.08227928 - samples/sec: 39.85
2019-09-11 14:42:30,465 epoch 56 - iter 12/21 - loss 0.08939202 - samples/sec: 39.35
2019-09-11 14:42:39,682 epoch 56 - iter 14/21 - loss 0.09158919 - samples/sec: 37.49
2019-09-11 

2019-09-11 14:51:09,957 epoch 61 - iter 16/21 - loss 0.09133365 - samples/sec: 39.01
2019-09-11 14:51:18,840 epoch 61 - iter 18/21 - loss 0.08928832 - samples/sec: 39.34
2019-09-11 14:51:30,168 epoch 61 - iter 20/21 - loss 0.08906342 - samples/sec: 29.79
2019-09-11 14:51:32,287 ----------------------------------------------------------------------------------------------------
2019-09-11 14:51:32,289 EPOCH 61 done: loss 0.0891 - lr 0.0003
2019-09-11 14:51:35,721 DEV : loss 0.34118664264678955 - score 0.9048
2019-09-11 14:51:35,815 BAD EPOCHS (no improvement): 4
2019-09-11 14:51:35,820 ----------------------------------------------------------------------------------------------------
train mode resetting embeddings
train mode resetting embeddings
2019-09-11 14:51:40,096 epoch 62 - iter 0/21 - loss 0.12120553 - samples/sec: 60.96
2019-09-11 14:51:49,057 epoch 62 - iter 2/21 - loss 0.08459720 - samples/sec: 36.58
2019-09-11 14:51:57,992 epoch 62 - iter 4/21 - loss 0.08949533 - samples/se

2019-09-11 15:00:23,531 epoch 67 - iter 6/21 - loss 0.08046204 - samples/sec: 37.66
2019-09-11 15:00:32,830 epoch 67 - iter 8/21 - loss 0.09162855 - samples/sec: 38.70
2019-09-11 15:00:41,785 epoch 67 - iter 10/21 - loss 0.09266437 - samples/sec: 38.73
2019-09-11 15:00:50,571 epoch 67 - iter 12/21 - loss 0.09091967 - samples/sec: 38.71
2019-09-11 15:00:58,737 epoch 67 - iter 14/21 - loss 0.08724763 - samples/sec: 38.06
2019-09-11 15:01:07,540 epoch 67 - iter 16/21 - loss 0.08183526 - samples/sec: 36.90
2019-09-11 15:01:16,443 epoch 67 - iter 18/21 - loss 0.08785178 - samples/sec: 37.90
2019-09-11 15:01:27,548 epoch 67 - iter 20/21 - loss 0.08954312 - samples/sec: 27.58
2019-09-11 15:01:29,416 ----------------------------------------------------------------------------------------------------
2019-09-11 15:01:29,418 EPOCH 67 done: loss 0.0895 - lr 0.0001
2019-09-11 15:01:33,151 DEV : loss 0.3412226140499115 - score 0.9048
2019-09-11 15:01:33,252 BAD EPOCHS (no improvement): 4
2019-09-11

{'test_score': 0.8661,
 'dev_score_history': [0.7292,
  0.8036,
  0.9107,
  0.9077,
  0.8899,
  0.9048,
  0.9018,
  0.7173,
  0.9018,
  0.9048,
  0.9018,
  0.8899,
  0.8988,
  0.8988,
  0.9018,
  0.9018,
  0.9018,
  0.9048,
  0.9018,
  0.9077,
  0.9077,
  0.8988,
  0.9048,
  0.8988,
  0.9048,
  0.9018,
  0.8958,
  0.8958,
  0.8958,
  0.9018,
  0.9077,
  0.9018,
  0.9048,
  0.9048,
  0.9018,
  0.9048,
  0.9048,
  0.9048,
  0.9048,
  0.9048,
  0.9048,
  0.9048,
  0.9018,
  0.9018,
  0.9048,
  0.9048,
  0.9048,
  0.9048,
  0.9048,
  0.9018,
  0.9048,
  0.9048,
  0.9048,
  0.9048,
  0.9048,
  0.9048,
  0.9048,
  0.9048,
  0.9048,
  0.9048,
  0.9048,
  0.9048,
  0.9048,
  0.9048,
  0.9048,
  0.9048,
  0.9048,
  0.9048,
  0.9048],
 'train_loss_history': [0.6854279651528313,
  0.515695877018429,
  0.38372806424186345,
  0.3911272571200416,
  0.24075230672245934,
  0.25772849718729657,
  0.20229697546788625,
  0.19222422334409894,
  0.26451183429786135,
  0.15876812842630206,
  0.1478492103162

In [ ]:
import torch
torch.cuda.is_available()

In [23]:
from flair.models import TextClassifier
from flair.data import Sentence

classifier = TextClassifier.load(result_loc+'/best-model.pt')


2019-09-11 15:09:53,706 loading file ../flair_models/text_classification/sentiment_model/best-model.pt


In [26]:
sentence = Sentence('The FCRL3 polymorphisms associated with rheumatoid arthritis in a Japanese population are not associated per se with rheumatoid arthritis in a Spanish population.')
# sentence = Sentence('We obtained evidence from two Caucasian study populations that the His298-allele of PTGES2 Arg298His confers to reduced risk of type 2 diabetes.')
# sentence = Sentence('it does not seem that variants in MnSOD, CAT, or GPX1 have an influence on prostate cancer risk in this cohort of men who were smokers or exposed to asbestos, although it is possible that cumulative defects in protection from oxidative stress may result in increased risk of the disease.')
classifier.predict(sentence)
print(sentence.labels)

[NA (0.9873950481414795)]


In [ ]:
from tqdm import tqdm

In [ ]:
   
with open(complete_file_path) as test:
    print(complete_file_path)
predicted_label = []
for index, rows in tqdm(df.iterrows(), total=df.shape[0]):
    if rows['label'] == 'NA':
       ground_truth.append('Negative')
    else:
       ground_truth.append('Positive')
    sentence = Sentence(rows['text'])
    classifier.predict(sentence)
    predicted_label.append(sentence.labels[0].value)
    
    